<a href="https://colab.research.google.com/github/gpenessot/OCT23_BDS_Radios_Poumons/blob/main/4_Transfer_learning_%26_hyperparameter_tuning_VGG19_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet DST - Etape 3 Modélisation - Transfer learning

Notebook adapté du notebook commun "baseline - hyperparameter tuning"

## 1) Montage Drive & import librairies : à faire à chaque ouverture

In [ ]:
# Connection du notebook au google drive perso

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Installation du keras tuner dans l'environnement du notebook car pas installé par défaut

!pip install keras-tuner -q
import keras_tuner
from keras_tuner.tuners import  Hyperband

In [ ]:
# Import des packages / librairies / fonctions nécessaires

import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import cv2
import zipfile
import pathlib

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam



from keras.callbacks import EarlyStopping

#Ajouts Thibaut :
import pickle #pour sauvegarde de l'historique d'entraînement
import os
import pandas as pd
#pour le Grad-CAM
from IPython.display import Image, display
import matplotlib
import matplotlib.cm as cm

## 2) Dézippage sets d'images : fait 1 fois

In [ ]:
# Dézippage des sets d'images (stockés sur google drive) dans l'environnement du notebook

# Dézippage train set
!unzip -q '/content/drive/MyDrive/Colab/TrainTest.zip'

## 3) Import des datasets

In [ ]:
#A MODIFIER PAR L'UTILISATEUR : Paramétrage des dossiers locaux pour les datasets et les sauvegardes :

# Chemin du dossier TRAIN dézippé : à adapter si stocké ailleurs
dossier_train = "/content/Train/"

# Chemin du dossier TEST dézippé : à adapter si stocké ailleurs
dossier_test = "/content/Test"

#Dossier de sauvegarde du modèle au format.keras et de l'historique d'entraînement au format pickle
dossier_sauv = "/content"

#Nom du dataset (arbitraire), pour intégrer dans le nom de la sauvegarde et s'y retrouver plus facilement
#N'a pas de rapport avec le nom du dossier
nom_dataset = "TrainTest"

In [ ]:
# Affichage du lien du répertoire du set d'entrainement
train_set_dir = pathlib.Path(dossier_train).with_suffix('')

# Vérification nombre d'images dans le set d'entrainement
print ("Répertoire train set : ", train_set_dir)
train_set_image_count = len(list(train_set_dir.glob('*/*.png')))
print("Nombre images train set ", train_set_image_count)

print ("\n")

# Affichage du lien du répertoire du set de test
test_set_dir = pathlib.Path(dossier_test).with_suffix('')

# Vérification nombre d'images dans le set de test
print ("Répertoire test set : ", test_set_dir)
test_set_image_count = len(list(test_set_dir.glob('*/*.png')))
print("Nombre images test set ", test_set_image_count)

## 3) Paramètres globaux

In [ ]:
IMG_HEIGHT = 224
IMG_WIDTH = 224
MODE = 'rgb'   # rgb or grayscale

AUGMENTATION_ON = False

NUM_EPOCHS = 50
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2
L_RATE = 0.001

NAME_TO_SAVE = "Resnet152v2_unfrozen"

keras_tuner_logs_dir = "keras_tuner/logs"

CALLBACKS = [
    tf.keras.callbacks.ModelCheckpoint(NAME_TO_SAVE+'.weights.h5', save_best_only=True,save_weights_only=True, monitor="val_accuracy"),
    tf.keras.callbacks.ReduceLROnPlateau( monitor="val_accuracy", factor=0.2, patience=6, min_lr=0.000000001),
    tf.keras.callbacks.TensorBoard('keras_tuner/log')]

In [ ]:
# Création des sets d'entrainement et de validation au format keras
# Attention : modèles de transfer learning requièrent du RGB en input

# Création du set d'entrainement
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
  train_set_dir,
  validation_split=VALIDATION_SPLIT,
  subset="both",
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  color_mode=MODE,
 # color_mode = "rgb", #pour les modèles de transfer_learning
  batch_size=BATCH_SIZE,
  label_mode= 'categorical')


# Création du set de validation
test_ds = tf.keras.utils.image_dataset_from_directory(
  train_set_dir,
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  color_mode=MODE,
#  color_mode = "rgb", #pour les modèles de transfer_learning
  batch_size=BATCH_SIZE,
  label_mode= 'categorical')

    #Configure dataset for performance with CACHE (not loading images at each epoch)
    #and PREFETCH (overlapping data processing and model execution)



## 4) Transfer learning sans tuning

### Import d'un modèle pré-entraîné pour le transfer learning

In [ ]:
#Import du modèle pré-entraîné et des poids sur un jeu de données
from tensorflow.keras.applications.resnet_v2 import ResNet152V2, preprocess_input, decode_predictions

#Attention, l'input standard de VGG16 est 224*224*3, il faudra ajouter une couche de resizing
#img_size_dataset = (img_height,img_width)
#img_size_model = (224, 224)

# Modèle VGG16 sans les couches denses de classification. Nécessite input en RGB
base_model = ResNet152V2(include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation=None)

# Freezer les couches du VGG16 pour ne pas les ré-entraîner
for layer in base_model.layers:
    layer.trainable = True

#Appliquer le preprocessing sur les inputs ("Note: each Keras Application expects a specific kind of input preprocessing. resnet_v2.preprocess_input will scale input pixels between -1 and 1
#La fonction preprocess_input est dépendante du modèle choisi (à importer avec le modèle en début de cellule)
def preprocess(images, labels):
  return preprocess_input(images), labels

train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)
#Attention, le préprocess réécrit sur train_ds et val_ds, il faut donc le réimporter si on change de modèle


In [ ]:
inputs = tf.keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
x = keras.layers.Rescaling(1./255)(inputs)
x = base_model(x)
x = layers.Flatten()(x)
x = layers.Dense(units=128, activation="tanh")(x)
x = layers.Dense(units=32, activation="tanh")(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(units=16, activation="tanh")(x)
x = layers.Dense(4, activation = "softmax")(x)


model = keras.Model(inputs, x)
model._name = NAME_TO_SAVE



### Compilation et entraînement

In [ ]:
#Compilation
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=[keras.metrics.CategoricalAccuracy(name ='accuracy')])
model.summary()

In [ ]:
# Définition d'une callback qui stoppe le processus d'entrainement quand une valeur monitorée
# (ici la validation loss) stagne pendant un certain nombre d'epochs (et ce à partir d'un certain nombre d'epochs)

CALLBACKS = [
    tf.keras.callbacks.ModelCheckpoint(NAME_TO_SAVE+'.weights.h5', save_best_only=True,save_weights_only=True, monitor="val_accuracy"),
    tf.keras.callbacks.ReduceLROnPlateau( monitor="val_accuracy", factor=0.2, patience=6, min_lr=0.000000001),
    tf.keras.callbacks.TensorBoard("keras_tuner/logs")]

In [ ]:
#Entraînement
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=NUM_EPOCHS,
                    callbacks = CALLBACKS )


###

### Affichage des résultats et Enregistrement du modèle et des résultats

In [ ]:
def plot_history_metrics(history,name_to_save):
 acc = history.history['accuracy']
 val_acc = history.history['val_accuracy']

 loss = history.history['loss']
 val_loss = history.history['val_loss']


 fig = plt.figure(figsize=(12,6))
 ax1 = fig.add_subplot(121)

 ax1.plot(acc, label='Training Accuracy')
 ax1.plot(val_acc, label='Validation Accuracy')
 ax1.legend(loc='lower right')
 ax1.set_title(name_to_save + 'Accuracy')

 ax2 = fig.add_subplot(122)
 ax2.plot(loss, label='Training Loss')
 ax2.plot(val_loss, label='Validation Loss')
 ax2.legend(loc='upper right')
 ax2.set_title( name_to_save + 'Loss')
 plt.show()

 fig.savefig('T&V' + name_to_save + '.png')
 plt.close(fig)

In [ ]:
#Sauvegarde du modèle et des résultats

#Sauvegarde du modèle
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1

model.save('/content/'+ NAME_TO_SAVE + '.keras')

#Export results to dict & csv

#Dump to a pickle file
# Pickle the history to file
filepath = f"{NAME_TO_SAVE}_history.pkl"
filepath = os.path.join(dossier_sauv, filepath)
with open(filepath, 'wb') as f:
    pickle.dump(history, f)

#Impression du schéma
plot_history_metrics(history, NAME_TO_SAVE )

In [ ]:
#Métriques sur jen d'entraînement
best_val_loss_index = np.argmin(history.history["val_loss"])
print("Best validation loss :", round(np.max(history.history["val_loss"]), 2) )
print("Validation ACCURACY : ", round(history.history["val_accuracy"][best_val_loss_index], 2))
print("Obtained at epoch : ", best_val_loss_index + 1)




## 6) Evaluer le modèle sur un jeu de test

In [ ]:
model.load_weights(NAME_TO_SAVE+'.weights.h5')

accuracy = model.evaluate(test_ds, return_dict=True)

print('TEST ACCURACY:',accuracy.get('accuracy'))

## 7) Interprétabilité Grad-CAM

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions



### Paramètres à configurer selon le modèle


In [ ]:
img_size = (256, 256)


model.layers[4].activation = None

last_conv_layer_name = "conv2d_7"



### Définition des fonctions



In [ ]:
#Fonction pour charger une nouvelle image comme array pour injecter dans la fonction GradCam
def get_img_array(img_path, size):
    # `img` is a PIL image of size 256*256
    img = keras.utils.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (256, 256, 3)
    array = keras.utils.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 256,256, 3)
    array = np.expand_dims(array, axis=0)
    return array

In [ ]:
#Fonction grad_cam
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = keras.models.Model(
        model.inputs, [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
def create_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.5):
    # Load the original image
    img = keras.utils.load_img(img_path)
    img = keras.utils.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = matplotlib.colormaps["jet"]

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.utils.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.utils.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    #display(Image(img_path))

    #Return superimposed_img
    return superimposed_img


### Test sur 1 image

In [ ]:
#Création de la heatmap pour l'image donnée, en utilisant la dernière couche de convolution du modèle

#Tirer et affichier une image au hasard dans le dataset Train
train_set_list = list(train_set_dir.glob('*/*.png'))
k = int(np.random.choice(len(train_set_list), 1))
img_path = str(train_set_list[k])
img = keras.utils.load_img(img_path)

#Récupérer le nom de l'image pour afficher la vraie classe
img_name = str(img_path).rsplit("/",1)[1]
real_class_name = img_name.split("-")[0]

# Prepare image
img_array = preprocess_input(get_img_array(img_path, size=(IMG_HEIGHT,IMG_WIDTH)))

# Print what the top predicted class is
preds = model.predict(img_array)

#Si le modèle est chargé avec sa couche de classification, utiliser la fonction standard de Keras :
#prediction = decode_predictions(preds, top=1)[0]

#Sinon, le nombre de classes est différent, on calcule à la main:
best_pred_index = preds.argmax()
prediction = train_ds.class_names[best_pred_index]

print(img_path)
print("Classe réelle : ", real_class_name)
print("Prédiction : ", prediction)

# Generate class activation heatmap
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)


# Display heatmap
plt.matshow(heatmap)
plt.show()



In [ ]:
#Afficher les images superposées ainsi que la prédiction

#Création de la gradcam : fonctions adaptées depuis https://keras.io/examples/vision/grad_cam/
gradcam = create_gradcam(img_path, heatmap)

plt.figure(figsize = (10, 5))

#Affichage de la classe réelle et prédite
print("Classe réelle : ",real_class_name)
#fonction pour décoder la prédiction à adapter
print("Classe prédite par le modèle : ", prediction)


#Affichage de l'image source
plt.subplot(121)
plt.imshow(img, cmap = "gray")
plt.title(img_name)
plt.axis("off")

#AFfichage de la grad-cam
plt.subplot(122)
plt.imshow(gradcam)
plt.axis("off")
plt.show()
